Four phases:
1. Read in the zipped json files from the s3 bucket
2. Establish the PSQL connection
3. Transform the data
4. Load the data

In [3]:
import requests
from io import BytesIO
from zipfile import ZipFile
import sys

import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import String, Boolean, BIGINT, DateTime, Float
import creds


import pandas as pd
import json
import datetime
from collections import Counter

# import os

/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Read in the zipped json files from the s3 bucket

In [4]:
def download_extract_zip(url):
    r=requests.get(url)
    if (r.status_code=='200'):
        print('Loading data...')
    with ZipFile(BytesIO(r.content)) as thezip:
        for zipinfo in thezip.infolist():
            with thezip.open(zipinfo) as thefile:
                yield zipinfo.filename, thefile

def main(args):
    print(args)
    url=args[0]
    datafiles=download_extract_zip(url)
    
    # Connecting to PostgreSQL by providing a sqlachemy engine
    engine = create_engine('postgresql://'+creds.username+':'
                           +creds.password+'@'+creds.host+':'
                           +creds.port+'/'+creds.db,echo=False)

if __name__ == "__main__":
    main(sys.argv)

In [ ]:
  
def convert_timestring_utc(x):
    if x is None:
        return x
    x=x[:-6]+''.join(x[-6:].split(':'))
    return datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S%z').astimezone(tz=datetime.timezone.utc)

def convert_datetime_to_str(x):
    if x is None:
        return x
    else:
        return x.strftime('%Y-%m-%d %H:%M:%S%z')

exclude_list=['closed_at','created_at','updated_at',
              'processed_at','cancelled_at','line_items',
             'id']
include_list=['email','number','total_price','confirmed']
user_include_list=['customer_locale','buyer_accepts_marketing','contact_email','phone']

In [ ]:
order_dict={}
line_item_dict={}
user_dict={}
for (fn,fobj) in datafiles:
    print(fn)
    data=json.load(fobj)
    orders=data['orders']
    for order in orders:
        order_id=order['id']
        line_item_list=order['line_items']
        line_item_total=len(line_item_list)
        item_total=sum([x['quantity'] for x in line_item_list])
        order_dict[order_id]={k:v for k,v in order.items() 
                              if( k not in set(exclude_list) 
                                 and k not in set(user_include_list))}
        order_dict[order_id]['line_item_total']=line_item_total
        for line_item in line_item_list:
            line_item_id=line_item['id']
            line_item_dict[line_item_id]={k:v for k,v in line_item.items() if k!='id'}
            line_item_dict[line_item_id]['order_id']=order_id
    break

In [ ]:
order_df=pd.DataFrame.from_dict(order_dict,orient='index')
order_df.head()

In [ ]:


order_type_dict={
    'email':String,
    'number':BIGINT,
    'total_price':Float,
    'total_tax':Float,
    'subtotal_price':Float,
    'total_discount':Float,
    'total_line_items_price':Float,
    'confirmed':Boolean,
    'checkout_id':BIGINT,
    'line_item_total':BIGINT,
    'processed_at':DateTime(timezone=True),
    'cancelled_at':DateTime(timezone=True),
    'created_at':DateTime(timezone=True),
    'updated_at':DateTime(timezone=True),
    'closed_at':DateTime(timezone=True)   
}

# order_df.to_sql(name='order_test2', con=engine, if_exists = 'append', index=True,dtype=order_type_dict)

In [ ]:
main:
    load the file from s3 link
    connect to psql
    